# --- Day 22: Monkey Map ---

[Puzzle Description](https://adventofcode.com/2022/day/22)

In [1]:
import numpy as np
import re

In [2]:
grid = list()
max_width = 0
symbol_dict = {" ": 0, ".": 1, "#": 9}
with open("day_22_input.txt") as file:
    # Create the grid
    while line := file.readline().rstrip():
        max_width = max(max_width, len(line))
        row = [symbol_dict[line[i]] if i < len(line) else 0 for i in range(max_width)]
        grid.append(row)
    grid = np.array(grid)

    # Save the path
    line = file.readline().rstrip()
    steps = [int(num) for num in re.split(r"[RL]", line)]
    rotations = [1 if c == "R" else -1 for c in re.split(r"[0-9]+", line)[1:-1]]

In [3]:
# Movement according to the current orientation:
movement = {
    0: np.array([0, 1]),
    1: np.array([1, 0]),
    2: np.array([0, -1]),
    3: np.array([-1, 0]),
}

# Grid limits for each direction:
grid_limits = {
    0: ((grid > 0) * [range(grid.shape[1]) for i in range(grid.shape[0])]).argmax(
        axis=1
    ),
    1: ((grid > 0) * [[i] * grid.shape[1] for i in range(grid.shape[0])]).argmax(
        axis=0
    ),
    2: (grid > 0).argmax(axis=1),
    3: (grid > 0).argmax(axis=0),
}

## Part One

In [4]:
# Setting the starting configuration:
facing = 0
cur_y = 0
cur_x = grid_limits[2][cur_y]  # Find the left limit of row 0
cur_pos = np.array([cur_y, cur_x])

# Run the steps:
for i in range(len(steps)):
    # Move for the length of the step:
    for j in range(steps[i]):
        # Compute the next position:
        next_pos = cur_pos + movement[facing]

        # Compute the wrapping if necessary:
        if facing == 0 and next_pos[1] > grid_limits[facing][next_pos[0]]:
            next_pos[1] = grid_limits[2][next_pos[0]]
        elif facing == 1 and next_pos[0] > grid_limits[facing][next_pos[1]]:
            next_pos[0] = grid_limits[3][next_pos[1]]
        elif facing == 2 and next_pos[1] < grid_limits[facing][next_pos[0]]:
            next_pos[1] = grid_limits[0][next_pos[0]]
        elif facing == 3 and next_pos[0] < grid_limits[facing][next_pos[1]]:
            next_pos[0] = grid_limits[1][next_pos[1]]

        # Stop moving if a wall is encountered
        if grid[tuple(next_pos)] == 9:
            break

        # Complete the move
        cur_pos = next_pos

    # Unless this is the last step, rotate accordingly:
    if i < len(rotations):
        facing = (facing + rotations[i]) % 4

cur_y, cur_x = cur_pos
part_1_ans = 1000 * (cur_y + 1) + 4 * (cur_x + 1) + facing
print(f"Part One: {part_1_ans}")

Part One: 117054


## Part Two

Note that the shape of the input is used to hardcode the solutions:

```
 ##
 #
##
#
```

To come up with such a hardcode solution, it is best to draw the grid and see, which points are joined in a cube!

In [5]:
# Setting the starting configuration:
facing = 0
cur_y = 0
cur_x = grid_limits[2][cur_y]  # Find the left limit of row 0
cur_pos = np.array([cur_y, cur_x])

# Run the steps:
for i in range(len(steps)):
    # Move for the length of the step:
    for j in range(steps[i]):
        # Compute the next position:
        next_pos = cur_pos + movement[facing]
        old_facing = facing

        # Compute the wrapping if necessary:

        # Wrapping to the right:
        if facing == 0 and next_pos[1] > grid_limits[facing][next_pos[0]]:
            if cur_pos[0] < 50:
                next_pos[0] = 149 - cur_pos[0]
                next_pos[1] = grid_limits[0][next_pos[0]]
                facing = 2
            elif cur_pos[0] < 100:
                next_pos[1] = 50 + cur_pos[0]
                next_pos[0] = grid_limits[1][next_pos[1]]
                facing = 3
            elif cur_pos[0] < 150:
                next_pos[0] = 149 - cur_pos[0]
                next_pos[1] = grid_limits[0][next_pos[0]]
                facing = 2
            elif cur_pos[0] < 200:
                next_pos[1] = cur_pos[0] - 100
                next_pos[0] = grid_limits[1][next_pos[1]]
                facing = 3
            else:
                print("ERROR!")

        # Wrapping to the bottom:
        elif facing == 1 and next_pos[0] > grid_limits[facing][next_pos[1]]:
            if cur_pos[1] < 50:
                next_pos[1] = 100 + cur_pos[1]
                next_pos[0] = grid_limits[3][next_pos[1]]
                facing = 1
            elif cur_pos[1] < 100:
                next_pos[0] = 100 + cur_pos[1]
                next_pos[1] = grid_limits[0][next_pos[0]]
                facing = 2
            elif cur_pos[1] < 150:
                next_pos[0] = cur_pos[1] - 50
                next_pos[1] = grid_limits[0][next_pos[0]]
                facing = 2
            else:
                print("ERROR!")

        # Wrapping to the left:
        elif facing == 2 and next_pos[1] < grid_limits[facing][next_pos[0]]:
            if cur_pos[0] < 50:
                next_pos[0] = 149 - cur_pos[0]
                next_pos[1] = grid_limits[2][next_pos[0]]
                facing = 0
            elif cur_pos[0] < 100:
                next_pos[1] = cur_pos[0] - 50
                next_pos[0] = grid_limits[3][next_pos[1]]
                facing = 1
            elif cur_pos[0] < 150:
                next_pos[0] = 149 - cur_pos[0]
                next_pos[1] = grid_limits[2][next_pos[0]]
                facing = 0
            elif cur_pos[0] < 200:
                next_pos[1] = cur_pos[0] - 100
                next_pos[0] = grid_limits[3][next_pos[1]]
                facing = 1
            else:
                print("ERROR!")

        # Wrapping to the top:
        elif facing == 3 and next_pos[0] < grid_limits[facing][next_pos[1]]:
            if cur_pos[1] < 50:
                next_pos[0] = cur_pos[1] + 50
                next_pos[1] = grid_limits[2][next_pos[0]]
                facing = 0
            elif cur_pos[1] < 100:
                next_pos[0] = 100 + cur_pos[1]
                next_pos[1] = grid_limits[2][next_pos[0]]
                facing = 0
            elif cur_pos[1] < 150:
                next_pos[1] = cur_pos[1] - 100
                next_pos[0] = grid_limits[1][next_pos[1]]
                facing = 3
            else:
                print("ERROR!")

        # Stop moving if a wall is encountered
        if grid[tuple(next_pos)] == 9:
            # Revert the facing to the previous position!
            facing = old_facing
            break

        # Complete the move
        cur_pos = next_pos

    # Unless this is the last step, rotate accordingly:
    if i < len(rotations):
        facing = (facing + rotations[i]) % 4

cur_y, cur_x = cur_pos
part_2_ans = 1000 * (cur_y + 1) + 4 * (cur_x + 1) + facing
print(f"Part Two: {part_2_ans}")

Part Two: 162096
